In [1]:
from emc.workflows.correct import init_emc_wf
from pathlib import Path
import nipype.pipeline.engine as pe
from nipype.interfaces import utility as niu
from dipy.data import get_fnames
from dmriprep.workflows.dwi.util import init_dwi_reference_wf
from dmriprep.interfaces.vectors import CheckGradientTable

#fdata, fbvals, fbvecs = get_fnames("sherbrooke_3shell")
#fdata, fbvals, fbvecs = get_fnames("stanford_hardi")
fdata = '/mnt/data/datalad/ds000206/sub-THP0001/ses-THP0001CCF1/dwi/sub-THP0001_ses-THP0001CCF1_acq-GD31_run-01_dwi.nii.gz'
fbvals = '/mnt/data/datalad/ds000206/sub-THP0001/ses-THP0001CCF1/dwi/sub-THP0001_ses-THP0001CCF1_acq-GD31_run-01_dwi.bval'
fbvecs = '/mnt/data/datalad/ds000206/sub-THP0001/ses-THP0001CCF1/dwi/sub-THP0001_ses-THP0001CCF1_acq-GD31_run-01_dwi.bvec'


In [2]:
# Create a workflow that can be added as a node to
# `~dmriprep.workflows.dwi.base.init_dwi_preproc_wf` and (for now) envelops
# `~dmriprep.workflows.dwi.util.init_dwi_reference_wf`
def build_emc_workflow(fdata, fbvals, fbvecs, work_dir='/tmp'):

    wf_emc = pe.Workflow(name='emc_correction')

    inputnode = pe.Node(niu.IdentityInterface(fields=['dwi_files', 'in_bval',
                                                      'in_bvec']),
        name='inputnode')

    inputnode.inputs.dwi_file = fdata
    inputnode.inputs.in_bval = fbvals
    inputnode.inputs.in_bvec = fbvecs

    emc_wf_node = init_emc_wf('emc_wf')

    wf_emc.connect([
        (inputnode, emc_wf_node, [('dwi_file', 'meta_inputnode.dwi_file'),
                                  ('in_bval', 'meta_inputnode.in_bval'),
                                  ('in_bvec', 'meta_inputnode.in_bvec')])
    ])

    wf_emc.base_dir = work_dir

    cfg = dict(execution={'stop_on_first_crash': False,
                          'parameterize_dirs': True, 'crashfile_format': 'txt',
                          'remove_unnecessary_outputs': False,
                          'poll_sleep_duration': 0, 'plugin': 'MultiProc'})
    for key in cfg.keys():
        for setting, value in cfg[key].items():
            wf_emc.config[key][setting] = value

    return wf_emc

In [3]:
wf = build_emc_workflow(fdata, fbvals, fbvecs)
res = wf.run(plugin='MultiProc')

210308-04:26:54,524 nipype.workflow INFO:
	 Workflow emc_correction settings: ['check', 'execution', 'logging', 'monitoring']
210308-04:26:54,555 nipype.workflow INFO:
	 Running in parallel.
210308-04:26:54,560 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 2 jobs ready. Free memory (GB): 28.23/28.23, Free processors: 32/32.
210308-04:26:54,631 nipype.workflow INFO:
	 [Job 0] Cached (emc_correction.emc_wf.patch2self_node).
210308-04:26:54,632 nipype.workflow INFO:
	 [Job 3] Cached (emc_correction.emc_wf.emc_vectors_node).
210308-04:26:54,634 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 3 jobs ready. Free memory (GB): 28.23/28.23, Free processors: 32/32.
210308-04:26:54,707 nipype.workflow INFO:
	 [Job 1] Cached (emc_correction.emc_wf.dwi_reference_wf.validate).
210308-04:26:54,709 nipype.workflow INFO:
	 [Job 2] Cached (emc_correction.emc_wf.split_dwis_node).
210308-04:26:54,711 nipype.workflow INFO:
	 [Job 22] Cached (emc_correction.emc_wf.extract_b0_node).
21030

210308-04:27:23,628 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 1 jobs ready. Free memory (GB): 28.03/28.23, Free processors: 31/32.
                     Currently running:
                       * emc_correction.emc_wf.mask_4d_node
210308-04:27:23,708 nipype.workflow INFO:
	 [MultiProc] Running 2 tasks, and 0 jobs ready. Free memory (GB): 27.83/28.23, Free processors: 30/32.
                     Currently running:
                       * emc_correction.emc_wf.b0_emc_wf.iterative_alignment_000.reg_000
                       * emc_correction.emc_wf.mask_4d_node
210308-04:27:26,997 nipype.workflow INFO:
	 [Job 18] Completed (emc_correction.emc_wf.mask_4d_node).
210308-04:27:26,998 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 1 jobs ready. Free memory (GB): 28.03/28.23, Free processors: 31/32.
                     Currently running:
                       * emc_correction.emc_wf.b0_emc_wf.iterative_alignment_000.reg_000
210308-04:27:27,77 nipype.workflow INFO:
	 

210308-04:27:49,441 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 0 jobs ready. Free memory (GB): 28.03/28.23, Free processors: 31/32.
                     Currently running:
                       * emc_correction.emc_wf.b0_emc_wf.iterative_alignment_002.reg_002
210308-04:27:54,707 nipype.workflow INFO:
	 [Job 36] Completed (emc_correction.emc_wf.b0_emc_wf.iterative_alignment_002.reg_002).
210308-04:27:54,708 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 4 jobs ready. Free memory (GB): 28.23/28.23, Free processors: 32/32.
210308-04:27:54,776 nipype.workflow INFO:
	 [MultiProc] Running 4 tasks, and 0 jobs ready. Free memory (GB): 27.43/28.23, Free processors: 28/32.
                     Currently running:
                       * emc_correction.emc_wf.b0_emc_wf.iterative_alignment_002.averaged_images
                       * emc_correction.emc_wf.b0_emc_wf.iterative_alignment_002.transforms_to_list
                       * emc_correction.emc_wf.match_transforms_no

210308-04:28:03,592 nipype.workflow INFO:
	 [Job 75] Completed (_b0_based_image_transforms9).
210308-04:28:03,595 nipype.workflow INFO:
	 [MultiProc] Running 31 tasks, and 0 jobs ready. Free memory (GB): 22.03/28.23, Free processors: 1/32.
                     Currently running:
                       * _b0_based_image_transforms30
                       * _b0_based_image_transforms29
                       * _b0_based_image_transforms28
                       * _b0_based_image_transforms27
                       * _b0_based_image_transforms26
                       * _b0_based_image_transforms25
                       * _b0_based_image_transforms24
                       * _b0_based_image_transforms23
                       * _b0_based_image_transforms22
                       * _b0_based_image_transforms21
                       * _b0_based_image_transforms20
                       * _b0_based_image_transforms19
                       * _b0_based_image_transforms18
                  

210308-04:28:06,231 nipype.workflow INFO:
	 [Job 67] Completed (_b0_based_image_transforms1).
210308-04:28:06,233 nipype.workflow INFO:
	 [Job 74] Completed (_b0_based_image_transforms8).
210308-04:28:06,234 nipype.workflow INFO:
	 [Job 85] Completed (_b0_based_image_transforms19).
210308-04:28:06,235 nipype.workflow INFO:
	 [MultiProc] Running 2 tasks, and 0 jobs ready. Free memory (GB): 27.83/28.23, Free processors: 30/32.
                     Currently running:
                       * _b0_based_image_transforms13
                       * _b0_based_image_transforms0
210308-04:28:06,303 nipype.workflow INFO:
	 [Job 66] Completed (_b0_based_image_transforms0).
210308-04:28:06,304 nipype.workflow INFO:
	 [Job 79] Completed (_b0_based_image_transforms13).
210308-04:28:06,306 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 28.23/28.23, Free processors: 32/32.
210308-04:28:06,378 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 0 jobs ready

...

210309-00:18:35,440 nipype.workflow INFO:
	 [Job 202] Completed (_register_to_predicted15).
210309-00:18:35,449 nipype.workflow INFO:
	 [MultiProc] Running 13 tasks, and 2 jobs ready. Free memory (GB): 2.23/28.23, Free processors: 19/32.
                     Currently running:
                       * _register_to_predicted27
                       * _register_to_predicted26
                       * _register_to_predicted25
                       * _register_to_predicted24
                       * _register_to_predicted23
                       * _register_to_predicted22
                       * _register_to_predicted21
                       * _register_to_predicted20
                       * _register_to_predicted19
                       * _register_to_predicted18
                       * _register_to_predicted17
                       * _register_to_predicted16
                       * _register_to_predicted14
210309-00:18:35,637 nipype.workflow INFO:
	 [MultiProc] Running 14 tasks

210309-00:19:34,539 nipype.workflow INFO:
	 [Job 208] Completed (_register_to_predicted21).
210309-00:19:34,542 nipype.workflow INFO:
	 [MultiProc] Running 7 tasks, and 0 jobs ready. Free memory (GB): 14.23/28.23, Free processors: 25/32.
                     Currently running:
                       * _register_to_predicted29
                       * _register_to_predicted28
                       * _register_to_predicted27
                       * _register_to_predicted26
                       * _register_to_predicted25
                       * _register_to_predicted24
                       * _register_to_predicted23
210309-00:19:37,489 nipype.workflow INFO:
	 [Job 211] Completed (_register_to_predicted24).
210309-00:19:37,492 nipype.workflow INFO:
	 [MultiProc] Running 6 tasks, and 0 jobs ready. Free memory (GB): 16.23/28.23, Free processors: 26/32.
                     Currently running:
                       * _register_to_predicted29
                       * _register_to_predic

210309-00:19:54,878 nipype.workflow INFO:
	 [MultiProc] Running 2 tasks, and 1 jobs ready. Free memory (GB): 27.83/28.23, Free processors: 30/32.
                     Currently running:
                       * emc_correction.emc_wf.dwi_model_emc_wf.calculate_cnr
                       * emc_correction.emc_wf.average_predicted_node
210309-00:19:54,980 nipype.workflow INFO:
	 [MultiProc] Running 3 tasks, and 0 jobs ready. Free memory (GB): 27.63/28.23, Free processors: 29/32.
                     Currently running:
                       * emc_correction.emc_wf.dwi_model_emc_wf.emc_report
                       * emc_correction.emc_wf.dwi_model_emc_wf.calculate_cnr
                       * emc_correction.emc_wf.average_predicted_node
210309-00:19:55,590 nipype.workflow INFO:
	 [Job 53] Completed (emc_correction.emc_wf.dwi_model_emc_wf.calculate_cnr).
210309-00:19:55,592 nipype.workflow INFO:
	 [MultiProc] Running 2 tasks, and 0 jobs ready. Free memory (GB): 27.83/28.23, Free processors:

210309-00:19:58,281 nipype.workflow INFO:
	 [Job 242] Completed (_motion_correct_images25).
210309-00:19:58,282 nipype.workflow INFO:
	 [Job 246] Completed (_motion_correct_images29).
210309-00:19:58,283 nipype.workflow INFO:
	 [MultiProc] Running 25 tasks, and 0 jobs ready. Free memory (GB): 23.23/28.23, Free processors: 7/32.
                     Currently running:
                       * _motion_correct_images30
                       * _motion_correct_images28
                       * _motion_correct_images27
                       * _motion_correct_images26
                       * _motion_correct_images24
                       * _motion_correct_images23
                       * _motion_correct_images22
                       * _motion_correct_images21
                       * _motion_correct_images20
                       * _motion_correct_images19
                       * _motion_correct_images18
                       * _motion_correct_images17
                       * _moti

210309-00:19:58,845 nipype.workflow INFO:
	 [Job 226] Completed (_motion_correct_images9).
210309-00:19:58,846 nipype.workflow INFO:
	 [Job 240] Completed (_motion_correct_images23).
210309-00:19:58,846 nipype.workflow INFO:
	 [Job 241] Completed (_motion_correct_images24).
210309-00:19:58,847 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 1 jobs ready. Free memory (GB): 28.03/28.23, Free processors: 31/32.
                     Currently running:
                       * emc_correction.emc_wf.dwi_model_emc_wf.emc_report
210309-00:19:58,943 nipype.workflow INFO:
	 [MultiProc] Running 2 tasks, and 0 jobs ready. Free memory (GB): 27.83/28.23, Free processors: 30/32.
                     Currently running:
                       * emc_correction.emc_wf.motion_correct_images
                       * emc_correction.emc_wf.dwi_model_emc_wf.emc_report
210309-00:19:59,308 nipype.workflow INFO:
	 [Job 51] Completed (emc_correction.emc_wf.motion_correct_images).
210309-00:19:59,309 nipy

In [ ]:
list(res.nodes())